In [1]:
import numpy as np 
import pandas as pd 
import os
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
import torch.optim as optim
from tqdm import tqdm
import matplotlib.pyplot as plt
%matplotlib inline
from fastai.basics import *
from fastai.callbacks import * 


import glob
import rawpy
import sys
from skimage.measure import compare_ssim, compare_psnr



In [2]:
from dirs import *

In [3]:
from SID_Unet_Model import *         #model classes
from SID_Dataset import *             #Pytorch DataSet



# Training

In [4]:
# takes a few minutes

train_ds = ImageDataset(train_ids)
val_ds = ImageDataset(val_ids)


In [5]:
#patch sizes

train_ds.ps=512
val_ds.ps = 512

In [6]:
# CUDA for PyTorch
use_cuda = torch.cuda.is_available()
device = torch.device("cuda:0" if use_cuda else "cpu")

criterion = nn.L1Loss()
net = Unet()
databunch = DataBunch.create(train_ds,val_ds, device=device, bs =4, num_workers = 3)

In [9]:
learn = Learner(databunch,net,loss_func = criterion)

# save the best model as we go (pick between val and train loss)

#monitor = 'trn_loss'      #'val_loss'
#learn.callbacks = [SaveModelCallback(learn, monitor,mode='min')]


## Training loops

In [19]:
#no validation loss calculation at the end of each epoch. Saves on training time.
#do not run if wanting to load a model

learn.data.valid_dl = None


In [1]:
lr=1e-4    # 1e-4 then 1e-5 

learn.fit(2000,lr)

In [2]:
lr=1e-5    # 1e-4 then 1e-5 

learn.fit(2000,lr)

In [22]:
learn.save('savedmodel')